In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

using Markdown

# Import some libraries from the JDP project
using JDP.BugRefs
using JDP.Trackers
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Trackers.Bugzilla  # Functions for accessing the Bugzilla API(s)
using JDP.Repository
using JDP.Functional

trackers = load_trackers()

# Bug Tag Propogation

This partially automates copying existing bug tags from one failed test result to another.

## Find untagged test failures

First we fetch the test results from OpenQA.

In [ ]:
# Get some job results from the openqa.suse.de (osd) OpenQA instance.
# Optional arguments (after the ';') like 'groupid' are passed to the OpenQA API
allres = Repository.fetch(OpenQA.TestResult, Vector, "osd"; refresh=false, groupid=116)

Then filter the results to only include fails from a particular build with no bug references.

In [ ]:
build = "137.1"

untagged = filter(allres) do res
    res.build == build &&
    occursin(r"failed", res.result) &&
    isempty(res.refs)
end

## Find existing bug references for tests

Now we search all the test results for bug tags that were added in other builds or architectures.

In [40]:
tagdict = Dict{String, Set{JDP.BugRefs.Ref}}(res.name => Set() for res in untagged)

for res in allres
    if haskey(tagdict, res.name) && length(res.refs) > 0
        push!(tagdict[res.name], res.refs...)
    end
end

tagdict

Dict{String,Set{JDP.BugRefs.Ref}} with 97 entries:
  "btrfs-012"               => Set(JDP.BugRefs.Ref[])
  "xfs-189"                 => Set(JDP.BugRefs.Ref[])
  "trinity"                 => Set(JDP.BugRefs.Ref[])
  "shutdown_ltp"            => Set(JDP.BugRefs.Ref[])
  "generic-320"             => Set(JDP.BugRefs.Ref[])
  "xfs-150"                 => Set(JDP.BugRefs.Ref[])
  "btrfs-122"               => Set(JDP.BugRefs.Ref[])
  "xfs-145"                 => Set(JDP.BugRefs.Ref[])
  "xfs-147"                 => Set(JDP.BugRefs.Ref[])
  "btrfs-171"               => Set(JDP.BugRefs.Ref[bsc#1120036])
  "xfs-027"                 => Set(JDP.BugRefs.Ref[])
  "generic-252"             => Set(JDP.BugRefs.Ref[])
  "xfs-059"                 => Set(JDP.BugRefs.Ref[])
  "btrfs-139"               => Set(JDP.BugRefs.Ref[bsc#1114393])
  "btrfs-152"               => Set(JDP.BugRefs.Ref[])
  "xfs-151"                 => Set(JDP.BugRefs.Ref[])
  "mcast6-group-same-group" => Set(JDP.BugRefs.Ref[])
  "generi

Optionally we can get a summary of the bug tags it has found from Bugzilla.

> TODO: look on Progress instead for poo refs

In [6]:
bses = Bugzilla.login("bsc");

User Name: rpalethorpe
Password: ········


In [ ]:
bugdict = Dict(name => [] for name in keys(tagdict))
for (k, v) in pairs(tagdict)
    for rf in v
        @info "Fetching $rf for $k"
        push!(bugdict[k], rf => Bugzilla.get_bug(bses, parse(Int, rf.id)))
    end
end

In [8]:
mdbuf = IOBuffer()

for (k, v) in pairs(bugdict)
    if isempty(v)
        continue
    end
    
    println(mdbuf, "- ", k)
    for (rf, bug) in v
        print(mdbuf, "   * ")
        show(mdbuf, MIME("text/markdown"), rf)
        print(mdbuf, " ")
        show(mdbuf, MIME("text/markdown"), Bugzilla.to_md(bug))
        println(mdbuf)
    end
    
end

seek(mdbuf, 0)
Markdown.parse(mdbuf)

  * btrfs-171

      * [bsc#1120036](https://apibugzilla.suse.com/show_bug.cgi?id=1120036) **P2 - High**(*Normal*) NEW: xfstests btrfs/171 fails on 4.12.14-25.23-default
  * btrfs-139

      * [bsc#1114393](https://apibugzilla.suse.com/show_bug.cgi?id=1114393) **P2 - High**(*Normal*) NEW: xfstests btrfs/139 fails only in ppc64le in btrfs
  * xfs-348

      * [bsc#1085909](https://apibugzilla.suse.com/show_bug.cgi?id=1085909) **P3 - Medium**(*Normal*) NEW: fstests with xfs on xfs/348 with xfs regression 4.12.14-14-default
  * generic-484

      * [bsc#1111466](https://apibugzilla.suse.com/show_bug.cgi?id=1111466) **P3 - Medium**(*Normal*) NEW: fstests: generic/484 fails with record lock is not preserved across execve(2)
  * btrfs-172

      * [bsc#1114270](https://apibugzilla.suse.com/show_bug.cgi?id=1114270) **P4 - Low**(*Normal*) NEW: xfstests btrfs/172 fails in btrfs
  * generic-521

      * [bsc#1119212](https://apibugzilla.suse.com/show_bug.cgi?id=1119212) **P1 - Urgent**(*Critical*) NEW: generic/091 has a regression fail in xfs
  * generic-471

      * [bsc#1120171](https://apibugzilla.suse.com/show_bug.cgi?id=1120171) **P3 - Medium**(*Normal*) NEW: xfstests generic/471 in btrfs fails on 4.12.14-25.23-default
  * generic-091

      * [bsc#1119212](https://apibugzilla.suse.com/show_bug.cgi?id=1119212) **P1 - Urgent**(*Critical*) NEW: generic/091 has a regression fail in xfs
  * btrfs-131

      * [bsc#1120051](https://apibugzilla.suse.com/show_bug.cgi?id=1120051) **P2 - High**(*Normal*) NEW: xfstests btrfs/131 fails on 4.12.14-25.23-default
  * btrfs-170

      * [poo#39839](https://progress.opensuse.org/issues/39839) **P3 - Medium**(*Minor*) RESOLVED: Wireless: cannot enter ascii keys in wireless network setup
      * [bsc#1120166](https://apibugzilla.suse.com/show_bug.cgi?id=1120166) **P2 - High**(*Normal*) NEW: xfstests btrfs/170 fails on 4.12.14-25.23-default
  * generic-388

      * [bsc#1085400](https://apibugzilla.suse.com/show_bug.cgi?id=1085400) **P3 - Medium**(*Normal*) IN_PROGRESS: xfstests generic/388 fails with xfs
  * fanotify09

      * [bsc#1110880](https://apibugzilla.suse.com/show_bug.cgi?id=1110880) **P5 - None**(*Normal*) NEW: LTP test fanotify09 fails in SLE12-SP3
  * btrfs-078

      * [bsc#1119089](https://apibugzilla.suse.com/show_bug.cgi?id=1119089) **P2 - High**(*Normal*) NEW: xfstests btrfs/078 fails on 4.12.14-25.22-default
  * generic-488

      * [bsc#1111469](https://apibugzilla.suse.com/show_bug.cgi?id=1111469) **P3 - Medium**(*Normal*) NEW: generic/488 fails by having many file descriptors referring to deleted files open on btrfs
  * generic-263

      * [bsc#1119212](https://apibugzilla.suse.com/show_bug.cgi?id=1119212) **P1 - Urgent**(*Critical*) NEW: generic/091 has a regression fail in xfs
  * generic-513

      * [bsc#1114278](https://apibugzilla.suse.com/show_bug.cgi?id=1114278) **P3 - Medium**(*Normal*) NEW: xfstests generic/513 fails in btrfs
  * fallocate05

      * [bsc#1099134](https://apibugzilla.suse.com/show_bug.cgi?id=1099134) **P3 - Medium**(*Normal*) NEW: Btrfs fallocate PUNCH*HOLE | KEEP*SIZE fails on filled up FS on ppc64le
  * generic-475

      * [bsc#1078009](https://apibugzilla.suse.com/show_bug.cgi?id=1078009) **P2 - High**(*Normal*) NEW: btrfs: soft lockup in __endio*write*update_ordered via xfstests generic/475
      * [bsc#1120167](https://apibugzilla.suse.com/show_bug.cgi?id=1120167) **P2 - High**(*Normal*) NEW: xfstests generic/475 triggered a Call Trace in BTRFS, kernel BUG at ../fs/btrfs/extent-tree.c:9906
  * getrlimit03

      * [bsc#1119409](https://apibugzilla.suse.com/show_bug.cgi?id=1119409) **P2 - High**(*Minor*) NEW: LTP getrlimit03 test fails on s390x probably due to returned limit size
      * [poo#44933](https://progress.opensuse.org/issues/44933) **P3 - Medium**(*Normal*) RESOLVED: can't downsize ntfs partition while installation
  * generic-520

      * [bsc#1120173](https://apibugzilla.suse.com/show_bug.cgi?id=1120173) **P3 - Medium**(*Normal*) NEW: xfstests generic/520 in btrfs fails on 4.12.14-25.23-default


If necessary some tags can be removed

In [47]:
modded = []

for (t, rf) in ["getrlimit03" => "poo#44933", 
                "btrfs-170" => "poo#39839", 
                "fanotify09" => "bsc#1110880"]
    delete!(tagdict[t], BugRefs.Ref(rf, trackers))
    push!(modded, tagdict[t])
end

modded

3-element Array{Any,1}:
 Set(JDP.BugRefs.Ref[bsc#1119409])
 Set(JDP.BugRefs.Ref[bsc#1120166])
 Set(JDP.BugRefs.Ref[])           

In [43]:
tagdict["getrlimit03"], tagdict["btrfs-170"]

(Set(JDP.BugRefs.Ref[bsc#1119409]), Set(JDP.BugRefs.Ref[bsc#1120166]))

## Posting back to OpenQA

Now we can post the results back to OpenQA. First we create a dictionary of jobs which we will post the bug tags to.

In [48]:
taggings = Dict()

for res in untagged
    for rf in tagdict[res.name]
        tags = get!(taggings, res.job.id, [])
        push!(tags, res.name => rf)
    end
end
        
taggings     

Dict{Any,Any} with 21 entries:
  2369906 => Any["btrfs-131"=>bsc#1120051, "btrfs-139"=>bsc#1114393]
  2369790 => Any["btrfs-170"=>bsc#1120166, "generic-475"=>bsc#1120167, "generic…
  2370439 => Any["generic-471"=>bsc#1120171, "generic-520"=>bsc#1120173]
  2369474 => Any["btrfs-170"=>bsc#1120166, "generic-475"=>bsc#1120167, "generic…
  2369479 => Any["generic-471"=>bsc#1120171, "generic-513"=>bsc#1114278, "gener…
  2369477 => Any["generic-263"=>bsc#1119212]
  2369796 => Any["btrfs-170"=>bsc#1120166, "generic-475"=>bsc#1120167, "generic…
  2369475 => Any["generic-091"=>bsc#1119212]
  2370492 => Any["xfs-348"=>bsc#1085909]
  2370491 => Any["btrfs-170"=>bsc#1120166, "generic-475"=>bsc#1120167, "generic…
  2373336 => Any["getrlimit03"=>bsc#1119409]
  2369852 => Any["fallocate05"=>bsc#1099134]
  2369783 => Any["btrfs-131"=>bsc#1120051, "btrfs-139"=>bsc#1114393]
  2369905 => Any["btrfs-078"=>bsc#1119089]
  2369833 => Any["generic-388"=>bsc#1085400]
  2369760 => Any["btrfs-171"=>bsc#1120036, "

In [ ]:
oqa = get_tracker(trackers, "osd")
ses = Trackers.ensure_login!(oqa)

for (jid, refs) in taggings
    mdbuf = IOBuffer()
    
    print(mdbuf, 
        "This is an automated message from [JDP](https://gitlab.suse.de/rpalethorpe/jdp)",
        "<br><br>",
        "The following bug tags have been propogated: <br>")
    for (name, rf) in refs
        print(mdbuf, name, ": ", rf, "<br>")
    end
    
    text = String(take!(mdbuf))
    
    @info "Posting comment to job $jid: \n$text"
    OpenQA.post_job_comment(ses, jid, text)
end

## Failures still missing a tag

In [52]:
filter(kv -> isempty(kv.second), pairs(tagdict)) |> keys

Base.KeySet for a Dict{String,Set{JDP.BugRefs.Ref}} with 78 entries. Keys:
  "btrfs-012"
  "xfs-189"
  "trinity"
  "shutdown_ltp"
  "generic-320"
  "xfs-150"
  "btrfs-122"
  "xfs-145"
  "xfs-147"
  "xfs-059"
  "xfs-027"
  "generic-252"
  "btrfs-152"
  "xfs-151"
  "mcast6-group-same-group"
  "generic-501"
  "bootloader_zkvm"
  "xfs-095"
  "xfs-146"
  "xfs-087"
  "xfs-168"
  "xfs-154"
  "xfs-075"
  "xfs-158"
  "generic-226"
  ⋮